<a href="https://colab.research.google.com/github/Abonvicine/ESTUDOS---Gerais/blob/main/Calcula_contornos_de_Mapa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
!apt-get -qq install python-cartopy python3-cartopy


In [52]:
!pip install --upgrade geopandas gspread

Requirement already up-to-date: geopandas in /usr/local/lib/python3.6/dist-packages (0.8.2)
Requirement already up-to-date: gspread in /usr/local/lib/python3.6/dist-packages (3.6.0)


In [53]:
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
import gspread
from shapely.geometry import Point, Polygon

In [54]:
import pandas as pd
import geopandas as gpd
import os

In [55]:
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()

gc = gspread.authorize(GoogleCredentials.get_application_default())

workbook = gc.open("Dados de mapa")
wsRESUMO = workbook.worksheet("Resumo INMET")
wsVALORES = workbook.worksheet("Valores INMET")

In [56]:
dados = pd.DataFrame(wsRESUMO.get_all_records())

In [57]:
dados

,lat,lng,Acc,Reservatório,Bacia
0,-15.79,-47.93,0.0,,
1,-16.64,-49.22,0.0,,
2,-17.75,-49.10,0.0,,
3,-10.19,-48.30,0.0,,
4,-12.62,-47.87,0.4,,
...,...,...,...,...,...
405,-22.58,-55.16,0.4,,
406,-21.45,-54.34,0.0,,
407,-18.07,-54.55,1.4,,
408,-20.47,-53.76,0.8,,


In [58]:
INMET = {"lat":[],"lng":[],"bacia":[]}

In [59]:
os.listdir("/content/Bacias")

['Paranapanema',
 'São Francisco',
 'Outras Sul',
 'Uruguai',
 'Paranaiba',
 'Tocantins',
 '.ipynb_checkpoints',
 'Parana',
 'Tiete',
 'Itaipu',
 'Grande',
 'Madeira',
 'Iguacu']

In [60]:
bacia = "Grande"

In [63]:
for bacia in os.listdir("/content/Bacias"):
    try:
        baseDir = f"/content/Bacias/{bacia}"
        print(f"####Analisando pontos para a bacia do {bacia}####")
    except ValueError:
        print(f"Não foi possível abrir arquivo da, continuando processo de leitura.")
        continue

    for Arquivo in os.listdir(baseDir):
        print(f"Conferindo pontos pertencentes a {Arquivo}")
        
        try:    
            df = pd.read_csv(os.path.join(baseDir,Arquivo))
        except ValueError:
            print(f"Não foi possível abrir arquivo da, continuando processo de leitura.")
        continue

        polyBacia = Polygon(df.values.tolist())

        for index, row in dados.iterrows():

            p = Point(row["lng"],row["lat"])

            if polyBacia.contains(p):           
                print(f"ponto {p} pertence a bacia {bacia}")
                INMET["lat"].append(row["lat"])
                INMET["lng"].append(row["lng"])
                INMET["bacia"].append(bacia)
            else:
                continue
print("Arquivos Classificados com sucesso!")

####Analisando pontos para a bacia do Paranapanema####
Conferindo pontos pertencentes a Rosana_SMAP2.0.bln
Conferindo pontos pertencentes a Maua.bln
Conferindo pontos pertencentes a Capivara_SMAP2.0.bln
Conferindo pontos pertencentes a Chavantes_SMAP2.0.bln
Conferindo pontos pertencentes a Jurumirim_SMAP2.0.bln
Conferindo pontos pertencentes a CanoasI_SMAP2.0.bln
####Analisando pontos para a bacia do São Francisco####
Conferindo pontos pertencentes a RetiroBaixo.bln
Conferindo pontos pertencentes a Queimado.bln
Conferindo pontos pertencentes a Inc_SaoFrancisco_SaoRomao.bln
Conferindo pontos pertencentes a IncrementalTresMarias.bln
Conferindo pontos pertencentes a Boqueirao.bln
Conferindo pontos pertencentes a SaoRomao.bln
####Analisando pontos para a bacia do Outras Sul####
Conferindo pontos pertencentes a UHE_14_Julho_INC.bln
Conferindo pontos pertencentes a UHE_Ernestina_ajust.bln
Conferindo pontos pertencentes a UHE_D_Francisca_INC.bln
Conferindo pontos pertencentes a GPSouza_ajust.

In [65]:
INMET

{'bacia': [], 'lat': [], 'lng': []}

In [64]:
dfINMET = pd.DataFrame.from_dict(INMET)
copyRange = "A1:C" + str(dfINMET["lat"].count() + 2)
print("copiando dados para " + copyRange)
wsVALORES.update(copyRange, [dfINMET.columns.values.tolist()]+dfINMET.values.tolist())

copiando dados para A1:C2


{'spreadsheetId': '1Safllrjdbdl09jtsvK3-1gqY7KnMoVeMxOk0hKBXWr0',
 'updatedCells': 3,
 'updatedColumns': 3,
 'updatedRange': "'Valores INMET'!A1:C1",
 'updatedRows': 1}